In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_squared_error
from scipy.interpolate import interp1d

import plotly.graph_objects as go
import plotly.express as px
import plotly.colors

import numpy as np
import pandas as pd

from mvf_bto.data_loading import load_data

## Loading Data

In [5]:
data_path = "/Users/anoushkabhutani/PycharmProjects/10701-mvf-bto/data/2017-05-12_batchdata_updated_struct_errorcorrect.mat"

In [6]:
data = load_data(file_path= data_path, num_cells= 3)

100%|██████████| 3/3 [00:54<00:00, 18.23s/it]


In [7]:
single_cell_data = data['b1c2']['cycles']

## Preprocessing Data

In [11]:
df_list = []
max_cycle = 1175
for cycle_key, time_series in tqdm.tqdm(single_cell_data.items()):
    cycle_num = int(cycle_key)
    if cycle_num<1:
        continue
    df = pd.DataFrame({'t': time_series['t'], 
                       'V': time_series['V'],
                       'temp': time_series['T'],
                       'I': time_series['I'],
                       'Qd': time_series['Qd'],
                      }
                       )
    # drop duplicates to be able to interpolate over capacity
    df = df.drop_duplicates(subset='Qd')
    
    
    # get discharge part of curve only (current is negative during discharge)
    df = df[df.I<-3.85]
    
    # normalize voltage and temperature using fixed thershold's to avoid data leakage
    df['V_norm'] = (df.V-1.9)/(3.5-1.9)
    df['T_norm'] = (df.temp-24)/(38-24)
    
    interp_df = pd.DataFrame()
    # use capacity as reference to interpolate over
    Q_eval = [0, 0.0125, 0.025, 0.075, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 0.85, 0.9, 0.95, 0.975, 0.98, 0.99, 1.0, 1.005, 1.01, 1.015, 1.02]
    interp_df['Q_eval'] = Q_eval
    fV = interp1d(x=df.Qd, y =df.V_norm, kind='quadratic', fill_value='extrapolate')
    interp_df['V_norm'] = fV(Q_eval)
    ft = interp1d(x=df.Qd, y =df.t, kind='quadratic', fill_value='extrapolate')
    interp_df['t'] = ft(Q_eval)
    fT = interp1d(x=df.Qd, y =df['T_norm'], kind='quadratic', fill_value='extrapolate')
    interp_df['T_norm'] = fT(Q_eval)
    
    interp_df['Cycle'] = [cycle_num/max_cycle for i in range(len(interp_df))]
    
    df_list.append(interp_df)

100%|██████████| 1176/1176 [00:11<00:00, 99.96it/s] 


## What does raw versus interpolated data look like?

In [13]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = df.Qd, y = df.V_norm, showlegend=True, mode="markers", name="Raw"))
fig.add_trace(go.Scatter(x = interp_df.Q_eval, y = interp_df.V_norm, showlegend=True, mode="markers+lines" , name="Interpolated"))

In [14]:
# multivariate data preparation
# TODO: multi output (temperature + voltage)
# TODO: multiple time steps in the future 
X_list, y_list = [], []

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
            # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix-1, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

for df in df_list:
    # define input sequence
    in_seq1 = df['T_norm'].values
    in_seq2 = df['Q_eval'].values
    in_seq3 = df['V_norm'].values
    out_seq = df['V_norm'].values
    
    # convert to [rows, columns] structure
    in_seq1 = in_seq1.reshape((len(in_seq1), 1))
    in_seq2 = in_seq2.reshape((len(in_seq2), 1))
    in_seq3 = in_seq3.reshape((len(in_seq3), 1))
    out_seq = out_seq.reshape((len(out_seq), 1))
    
    # horizontally stack columns
    dataset = np.hstack((in_seq1, in_seq2, in_seq3, out_seq))
    
    # choose a number of time steps (for input window)
    n_steps = 4
    
    # convert into input/output
    X_cycle, y_cycle = split_sequences(dataset, n_steps)
    X_list.append(X_cycle)
    y_list.append(y_cycle)

In [18]:
batch_size = X_cycle.shape[0] 
# batch_size must be equal to the length of 1 input curve
# since for a stateful LSTM the cell state is cleared after a batch
# (look at the keras docs)
# we could write over own custom callback if we need batch_size != sequence_length
# (for the use t prediction as t+1 input case)
# but I'm not sure if that's acceptable practice

window_length = X_cycle.shape[1]
n_features = X_cycle.shape[2]

In [19]:
X_train = np.array([item for index, item in enumerate(X_list) if index % 100 != 0])
X_test = np.array([item for index, item in enumerate(X_list) if index % 100 == 0])

y_train = np.array([item for index, item in enumerate(y_list) if index % 100 != 0])
y_test = np.array([item for index, item in enumerate(y_list) if index % 100 == 0])

y_test = y_test.flatten()
y_train = y_train.flatten()

X_train = X_train.reshape(X_train.shape[0]*batch_size, X_train[0].shape[1] , X_train.shape[-1])
X_test = X_test.reshape(X_test.shape[0]*batch_size,X_test[0].shape[1], X_test.shape[-1])

In [20]:
sample_weight = np.ones(y_train.shape)

# TODO: experiment with different sample weights and thersholds
# (this is a arbitary guess)
sample_weight[y_train<0.6]=2
sample_weight[y_train<0.5]=3

In [21]:
# define model
# TODO: hyperparameter tuning (Anoushka)
model = Sequential()
model.add(LSTM(32, return_sequences=True, stateful=True, batch_input_shape=(19, 3, 3)))
model.add(LSTM(16, return_sequences=False))
model.add(Dense(32, activation="sigmoid"))
model.add(Dense(8))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')

In [22]:
# TODO: add validation set or validation split + early stopping
history = model.fit(X_train, y_train, 
                    epochs=50, 
                    batch_size=batch_size, 
                    shuffle=False, 
                    verbose=1, 
                    sample_weight=sample_weight)

Epoch 1/50
1163/1163 [==============================] - 8s 4ms/step - loss: 0.0052
Epoch 2/50
1163/1163 [==============================] - 5s 4ms/step - loss: 0.0020
Epoch 3/50
1163/1163 [==============================] - 4s 4ms/step - loss: 0.0018
Epoch 4/50
1163/1163 [==============================] - 4s 4ms/step - loss: 0.0015
Epoch 5/50
1163/1163 [==============================] - 4s 4ms/step - loss: 0.0013
Epoch 6/50
1163/1163 [==============================] - 5s 4ms/step - loss: 0.0012
Epoch 7/50
1163/1163 [==============================] - 4s 4ms/step - loss: 0.0010 ETA: 0s
Epoch 8/50
1163/1163 [==============================] - 4s 4ms/step - loss: 0.0015
Epoch 9/50
1163/1163 [==============================] - 5s 4ms/step - loss: 6.6169e-04
Epoch 10/50
1163/1163 [==============================] - 5s 4ms/step - loss: 5.3941e-04
Epoch 11/50
1163/1163 [==============================] - 5s 4ms/step - loss: 4.5801e-04
Epoch 12/50
1163/1163 [==============================] - 4s 4ms/s

In [28]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = np.linspace(1,50), y = history.history['loss'],
                         showlegend=False, mode="markers+lines"))
fig.update_xaxes(title='Epochs')
fig.update_yaxes(title='Loss (MSE)')

## Parity Plot of Training Error

In [31]:
# random plotting traing error at some interval = skip to not make the plot rendering too slow
skip = 20

fig = go.Figure()
fig.add_trace(go.Scatter(x = [0,1], y = [0,1], showlegend=False, mode="markers+lines"))
for i in range(0,len(X_train), batch_size*skip):
    fig.add_trace(go.Scatter(x = model.predict(X_train[i:i+batch_size]).flatten(), 
                             y = y_train[i:i+batch_size], 
                             showlegend=False, mode="markers+lines"))
fig.update_yaxes(title='Normalized Target')
fig.update_xaxes(title='Normalized Prediction')

## Parity Plot of Test Error

In [33]:
pallete = plotly.colors.qualitative.Dark24*10

fig = go.Figure()
fig.add_trace(go.Scatter(x = [0,1], y = [0,1], showlegend=False, mode="markers+lines"))

for i in range(0, len(X_test), batch_size):
    fig.add_trace(go.Scatter(x = model.predict(X_test[i:i+batch_size]).flatten(), 
                             y = y_test[i:i+batch_size], 
                             showlegend=False, mode="markers+lines", 
                             line_color = pallete[i]))
fig.update_yaxes(title='Normalized Target')
fig.update_xaxes(title='Normalized Prediction')

In [38]:
fig = go.Figure()
for i in range(0, len(X_test), batch_size):
    V_actual = y_test[i:i+batch_size]
    V_pred = model.predict(X_test[i:i+batch_size]).flatten()
    fig.add_trace(go.Scatter(x = Q_eval, y = V_actual*(3.5-1.9)+1.9, 
                             mode='lines', name = 'data', 
                             line_color = pallete[i]))
    fig.add_trace(go.Scatter(x = Q_eval, y = V_pred*(3.5-1.9)+1.9,
                             mode='markers', name = 'prediction', 
                             line_color = pallete[i]))
fig.update_yaxes(title="Voltage [V]")
fig.update_xaxes(title="Capacity [Ah]")